In [1]:
import numpy as np
import pandas as pd

import cobra
from cameo import pfba
from cameo.flux_analysis.simulation import room, lmoma

import escher

## Define IDs of knocked-out reactions

In [39]:
knockouts = [
    {"gene": "fbaA", "id": "FBA"},
    {"gene": "fbaB", "id": "FBA"},
    {"gene": "fbp", "id": "FBP"},
    {"gene": "gnd", "id": "GND"},
    {"gene": "pfkA", "id": "PFK"},
    {"gene": "pfkB", "id": "PFK"},
    {"gene": "pgi", "id": "PGI"},
    {"gene": "pgl", "id": "PGL"},
    {"gene": "ppsA", "id": "PPS"},
    {"gene": "pts", "id": "GLCptspp"},
    {"gene": "pykA", "id": "PYK"},
    {"gene": "pykF", "id": "PYK"},
    {"gene": "rpe", "id": "RPE"},
    {"gene": "rpiA", "id": "RPI"},
    {"gene": "rpiB", "id": "RPI"},
    {"gene": "sdhCD", "id": "SUCDi"},
    {"gene": "sucAB", "id": "AKGDH"},
    {"gene": "talA", "id": "TALA"},
    {"gene": "tktA", "id": "TKT1"},
    {"gene": "tktB", "id": "TKT1"},
    {"gene": "tpi", "id": "TPI"},
    {"gene": "zwf", "id": "G6PDH2r"},
]

## Dilution simulation

### iML1515 model

In [2]:
model = cobra.io.load_json_model('../../../DataAnalysis/DataIntegrationProject/models/iML1515.json')

In [19]:
# Minimize glucose consumption from medium - equiv to max exchange. 
model.objective = "EX_glc__D_e"
model.objective_direction = "max"
# make glucose uptake to be through Pts
model.reactions.GLCptspp.bounds = (0.0,1000)

# Simulate D 0.2 h-1
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = (0.19,0.21)
# Glucose uptake rate should more or less match Ishii, 2007
# GUR for WT @ aerobic glucose = 2.87 mmol/h/gDW
model.reactions.GLCptspp.bounds = (2.86,2.88)

dilution_result = pfba(model)
all_data = pd.DataFrame()

In [20]:
# write WT information
df = pd.DataFrame(dilution_result.fluxes)
df.index = df.index.rename("ID")
df = df.reset_index()
df = df.assign(sample_id="WT")
        
# rename some columns to be more compatible with other simulations
df = df.assign(author = "iML1515")
df = df.assign(BiGG_ID = df.ID)
df = df.rename({"fluxes" : "flux"}, axis = 1)
        
# calculate normalized fluxes 
glucose_uptake = -1 * df[df['ID'] == 'EX_glc__D_e']['flux'].values[0]
df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)

df.to_csv('../data/simulation_results/COBRA/iML1515/WT_02.csv')

In [21]:
all_data = pd.concat([all_data, df])

In [22]:
for reaction in knockouts:
    with model:
        ko_gene = reaction["gene"]
        # Simulate knockout
        model.reactions.get_by_id(reaction["id"]).knock_out()
        lmoma_result = lmoma(model, reference=dilution_result.fluxes)

        df = pd.DataFrame(lmoma_result.fluxes)
        df.index = df.index.rename("ID")
        df = df.reset_index()
        df = df.assign(sample_id=reaction["gene"])
        
        # rename some columns to be more compatible with other simulations
        df = df.assign(author = "iML1515")
        df = df.assign(BiGG_ID = df.ID)
        df = df.rename({"fluxes" : "flux"}, axis = 1)
        
        # calculate normalized fluxes 
        glucose_uptake = -1 * df[df['ID'] == 'EX_glc__D_e']['flux'].values[0]
        df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)
        
        df.to_csv(f"../data/simulation_results/COBRA/delta_{ko_gene}.csv")

        all_data = pd.concat([all_data, df], sort=False)

In [23]:
all_data.to_csv("../data/simulation_results/COBRA/iML1515/knockouts_all.csv")

In [ ]:
# escher.Builder(
#     map_name="e_coli_core.Core metabolism",
#     model=model,
#     reaction_data=dilution_result.fluxes.to_dict(),
# ).display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [12/Feb/2019 15:33:14] "GET / HTTP/1.1" 200 -


## Make calculations for the e_coli_core model

In [13]:
model = cobra.io.load_json_model('../../../DataAnalysis/DataIntegrationProject/models/e_coli_core.json')

In [4]:
model.objective = "EX_glc__D_e"
model.objective_direction = "max"

# Simulate D 0.2 h-1
model.reactions.BIOMASS_Ecoli_core_w_GAM.bounds = (0.19,0.21)
# Glucose uptake rate should more or less match Ishii, 2007
# GUR for WT @ aerobic glucose = 2.87 mmol/h/gDW
model.reactions.GLCpts.bounds = (2.86,2.88)

dilution_result = pfba(model)
all_data = pd.DataFrame()

In [5]:
# write WT information
df = pd.DataFrame(dilution_result.fluxes)
df.index = df.index.rename("ID")
df = df.reset_index()
df = df.assign(sample_id="WT")
        
# rename some columns to be more compatible with other simulations
df = df.assign(author = "Ec_core")
df = df.assign(BiGG_ID = df.ID)
df = df.rename({"fluxes" : "flux"}, axis = 1)
        
# calculate normalized fluxes 
glucose_uptake = -1 * df[df['ID'] == 'EX_glc__D_e']['flux'].values[0]
df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)

df.to_csv('../data/simulation_results/COBRA/core_model/WT_02.csv')

In [6]:
all_data = pd.concat([all_data, df])

In [11]:
for reaction in knockouts:
    with model:        
        ko_gene = reaction["gene"]
        print(f"Worinkg on gene {ko_gene}")
        try:
            # Simulate knockout, it may not grow at all
            model.reactions.get_by_id(reaction["id"]).knock_out()
            lmoma_result = lmoma(model, reference=dilution_result.fluxes)
            df = pd.DataFrame(lmoma_result.fluxes)
        except:
            df = pd.DataFrame(dilution_result.fluxes)
            df.fluxes = 0
                   
        df.index = df.index.rename("ID")
        df = df.reset_index()
        df = df.assign(sample_id=reaction["gene"])
        
        # rename some columns to be more compatible with other simulations
        df = df.assign(author = "Ec_core")
        df = df.assign(BiGG_ID = df.ID)
        df = df.rename({"fluxes" : "flux"}, axis = 1)
        
        # calculate normalized fluxes 
        glucose_uptake = -1 * df[df['ID'] == 'EX_glc__D_e']['flux'].values[0]
        df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)
        
        df.to_csv(f"../data/simulation_results/COBRA/core_model/delta_{ko_gene}.csv")

        all_data = pd.concat([all_data, df], sort=False)

Worinkg on gene fbaA
Worinkg on gene fbaB
Worinkg on gene fbp
Worinkg on gene gnd
Worinkg on gene pfkA
Worinkg on gene pfkB
Worinkg on gene pgi
Worinkg on gene pgl
Worinkg on gene ppsA
Worinkg on gene pts
Worinkg on gene pykA
Worinkg on gene pykF
Worinkg on gene rpe
Worinkg on gene rpiA
Worinkg on gene rpiB
Worinkg on gene sdhCD
Worinkg on gene sucAB
Worinkg on gene talA
Worinkg on gene tktAB
Worinkg on gene tktB
Worinkg on gene tpi
Worinkg on gene zwf


In [12]:
core_all_data.to_csv("../data/simulation_results/COBRA/core_model/knockouts_all.csv")

## Make calculations for the EColiCore2 model

In [34]:
model = cobra.io.read_sbml_model('../../../DataAnalysis/DataIntegrationProject/models/EColiCore2_compressed_bigg_names.sbml')

Adding exchange reaction EX_Biomass for boundary metabolite: Biomass
Adding exchange reaction EX_4CRSOL_ex for boundary metabolite: 4CRSOL_ex
Adding exchange reaction EX_5DRIB_ex for boundary metabolite: 5DRIB_ex
Adding exchange reaction EX_ac_ex for boundary metabolite: ac_ex
Adding exchange reaction EX_adp_c for boundary metabolite: adp_c
Adding exchange reaction EX_AMOB_ex for boundary metabolite: AMOB_ex
Adding exchange reaction EX_ca2_ex for boundary metabolite: ca2_ex
Adding exchange reaction EX_cl_ex for boundary metabolite: cl_ex
Adding exchange reaction EX_co2_ex for boundary metabolite: co2_ex
Adding exchange reaction EX_coa_c for boundary metabolite: coa_c
Adding exchange reaction EX_cobalt2_ex for boundary metabolite: cobalt2_ex
Adding exchange reaction EX_cu2_ex for boundary metabolite: cu2_ex
Adding exchange reaction EX_etoh_ex for boundary metabolite: etoh_ex
Adding exchange reaction EX_fe2_ex for boundary metabolite: fe2_ex
Adding exchange reaction EX_fe3_ex for boundar

In [35]:
# Minimize glucose consumption from medium - equiv to max exchange. 
model.objective = "EX_glc_DASH_D_ex"
model.objective_direction = "max"

# Simulate D 0.2 h-1
model.reactions.Growth.bounds = (0.19,0.21)
# Glucose uptake rate should more or less match Ishii, 2007
# GUR for WT @ aerobic glucose = 2.87 mmol/h/gDW
model.reactions.GLCptspp.bounds = (2.86,2.88)

dilution_result = pfba(model)
all_data = pd.DataFrame()

In [36]:
# write WT information
df = pd.DataFrame(dilution_result.fluxes)
df.index = df.index.rename("ID")
df = df.reset_index()
df = df.assign(sample_id="WT")
        
# rename some columns to be more compatible with other simulations
df = df.assign(author = "ECC2")
df = df.assign(BiGG_ID = df.ID)
df = df.rename({"fluxes" : "flux"}, axis = 1)
        
# calculate normalized fluxes 
glucose_uptake = -1 * df[df['ID'] == 'EX_glc_DASH_D_ex']['flux'].values[0]
df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)

df.to_csv('../data/simulation_results/COBRA/ECC2/WT_02.csv')

In [37]:
all_data = pd.concat([all_data, df])

In [38]:
for reaction in knockouts:
    with model:        
        ko_gene = reaction["gene"]
        print(f"Working on gene {ko_gene}")
        try:
            # Simulate knockout, it may not grow at all
            model.reactions.get_by_id(reaction["id"]).knock_out()
            lmoma_result = lmoma(model, reference=dilution_result.fluxes)
            df = pd.DataFrame(lmoma_result.fluxes)
        except:
            df = pd.DataFrame(dilution_result.fluxes)
            df.fluxes = 0
                   
        df.index = df.index.rename("ID")
        df = df.reset_index()
        df = df.assign(sample_id=reaction["gene"])
        
        # rename some columns to be more compatible with other simulations
        df = df.assign(author = "ECC2")
        df = df.assign(BiGG_ID = df.ID)
        df = df.rename({"fluxes" : "flux"}, axis = 1)
        
        # calculate normalized fluxes 
        glucose_uptake = -1 * df[df['ID'] == 'EX_glc_DASH_D_ex']['flux'].values[0]
        df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)
        
        df.to_csv(f"../data/simulation_results/COBRA/ECC2/delta_{ko_gene}.csv")

        all_data = pd.concat([all_data, df], sort=False)

Working on gene fbaA
Working on gene fbaB
Working on gene fbp
Working on gene gnd
Working on gene pfkA
Working on gene pfkB
Working on gene pgi
Working on gene pgl
Working on gene ppsA
Working on gene pts
Working on gene pykA
Working on gene pykF
Working on gene rpe
Working on gene rpiA
Working on gene rpiB
Working on gene sdhCD
Working on gene sucAB
Working on gene talA
Working on gene tktAB
Working on gene tktB
Working on gene tpi
Working on gene zwf


In [40]:
all_data.to_csv("../data/simulation_results/COBRA/ECC2/knockouts_all.csv")